# Repo Link
https://github.com/mr-kelsey/fa25-aai520-group6.git

# Objective
Build an autonomous Investment Research Agent that:
1. Plans its research steps for a given stock symbol.
2. Uses tools dynamically (APIs, datasets, retrieval).
3. Self-reflects to assess the quality of its output.
4. Learns across runs (e.g., keeps brief memories or notes to improve future analyses).

# Code

In [5]:
"""Imports"""
from dotenv import dotenv_values
from huggingface_hub import login
from pathlib import Path
from smolagents import CodeAgent, TransformersModel
from yaml import safe_load

from tools import yfinance

In [2]:
"""login to hugging face [needed for Llama model]"""
env = dotenv_values(".env")
login(token=env["auth"])

In [3]:
"""Set up initial agent"""
tools = yfinance.tools
model1 = TransformersModel(model_id="HuggingFaceTB/SmolLM2-1.7B-Instruct", device_map="auto", max_new_tokens=1000)
model2 = TransformersModel(model_id="meta-llama/Llama-3.2-1B-Instruct", device_map="auto", max_new_tokens=1000)
with open(Path("prompts", "prompts.yaml")) as stream:
    prompt_templates = safe_load(stream)

agent = CodeAgent(tools, model1, prompt_templates) # add_base_tools=True, planning_interval=3)

In [ ]:
task = "Retrieve the current price of amazon stock"

agent.run(f"""
You are an expert at navigating the Yahoo Finance API.  You always start by retrieving a Ticker
object for the stock you are looking at.  From there, you use the `ticker_*` tools to extract 
specific information about the stock.  If you do not know the ticker_symbol for a stock, you 
should ask the user.  Setting the `period` to `1d` while calling the `ticker_history` tool is your
preffered way to get the current stock price.

{task}
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You are an expert at navigating the Yahoo Finance API.  You always start by retrieving a Ticker                 │
│ object for the stock you are looking at.  From there, you use the `ticker_*` tools to extract                   │
│ specific information about the stock.  If you do not know the ticker_symbol for a stock, you                    │
│ should ask the user.  Setting the `period` to `1d` while calling the `ticker_history` tool is your              │
│ preffered way to get the current stock price.                                                                   │
│                                                                                                                 │
│ Retrieve the current price of amazon stock                                                                      │
│                                                                                                                 │
╰─ TransformersModel - HuggingFaceTB/SmolLM2-1.7B-Instruct ───────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  amazon_ticker = ticker(ticker_symbol="AMZN")                                                                     
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out: yfinance.Ticker object <AMZN>

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer(amazon_current_price)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Date
2025-09-22 00:00:00-04:00    227.630005
Name: Close, dtype: float64

[Step 4: Duration 50.60 seconds| Input tokens: 11,772 | Output tokens: 205]

Date
2025-09-22 00:00:00-04:00    227.630005
Name: Close, dtype: float64

## Prompt Chaining:
### Ingest News

### Preprocess

### Classify

### Extract

### Summarize

## Routing:

In [ ]:
"""Direct content to the right specialist (e.g., earnings, news, or market analyzers)."""
# Example from Lecture:
	# Pull earnings report
	# Search for CEO interviews
	# ask follow up questions if unclear
	# stops when confident in recommendation

## Evaluator–Optimizer:
### Generate analysis

### Evaluate quality

### Refine using feedback

# Summary

* Agent Design and Workflows
* Agent Functions and Capabilities
* Evaluation and Iteration